# CPSC 533V: Assignment 3 - Behavioral Cloning and Deep Q Learning

## 48 points total (9% of final grade)

## Hamidreza Aftabi 41655473
## Amin MohammadiNasrabadi 28486827

---
This assignment will help you transition from tabular approaches, topic of HW 2, to deep neural network approaches. You will implement the [Atari DQN / Deep Q-Learning](https://arxiv.org/abs/1312.5602) algorithm, which arguably kicked off the modern Deep Reinforcement Learning craze.

In this assignment we will use PyTorch as our deep learning framework.  To familiarize yourself with PyTorch, your first task is to use a behavior cloning (BC) approach to learn a policy.  Behavior cloning is a supervised learning method in which there exists a dataset of expert demonstrations (state-action pairs) and the goal is to learn a policy $\pi$ that mimics this expert.  At any given state, your policy should choose the same action the export would.

Since BC avoids the need to collect data from the policy you are trying to learn, it is relatively simple. 
This makes it a nice stepping stone for implementing DQN. Furthermore, BC is relevant to modern approaches---for example its use as an initialization for systems like [AlphaGo][go] and [AlphaStar][star], which then use RL to further adapte the BC result.  

<!--

I feel like this might be better suited to going lower in the document:

Unfortunately, in many tasks it is impossible to collect good expert demonstrations, making

it's not always possible to have good expert demonstrations for a task in an environemnt and this is where reinforcement learning comes handy. Through the reward signal retrieved by interacting with the environment, the agent learns by itself what is a good policy and can learn to outperform the experts.

-->

Goals:
- Famliarize yourself with PyTorch and its API including models, datasets, dataloaders
- Implement a supervised learning approach (behavioral cloning) to learn a policy.
- Implement the DQN objective and learn a policy through environment interaction.

[go]:  https://deepmind.com/research/case-studies/alphago-the-story-so-far
[star]: https://deepmind.com/blog/article/alphastar-mastering-real-time-strategy-game-starcraft-ii

## Submission information

- Complete the assignment by editing and executing the associated Python files.
- Copy and paste the code and the terminal output requested in the predefined cells on this Jupyter notebook.
- When done, upload the completed Jupyter notebook (ipynb file) on canvas.

## Task 0: Preliminaries

### PyTorch

If you have never used PyTorch before, we recommend you follow this [60 Minutes Blitz][blitz] tutorial from the official website. It should give you enough context to be able to complete the assignment.


**If you have issues, post questions to Piazza**

### Installation

To install all required python packages:

```
python3 -m pip install -r requirements.txt
```

### Debugging


You can include:  `import ipdb; ipdb.set_trace()` in your code and it will drop you to that point in the code, where you can interact with variables and test out expressions.  We recommend this as an effective method to debug the algorithms.


[blitz]: https://pytorch.org/tutorials/beginner/deep_learning_60min_blitz.html

## Task 1: Behavioral Cloning

Behavioral Cloning is a type of supervised learning in which you are given a dataset of expert demonstrations tuple $(s, a)$ and the goal is to learn a policy function $\hat a = \pi(s)$, such that $\hat a = a$.

The optimization objective is $\min_\theta D(\pi(s), a)$ where $\theta$ are the parameters the policy $\pi$, in our case the weights of a neural network, and where $D$ represents some difference between the actions.

---

Before starting, we suggest reading through the provided files.

For Behavioral Cloning, the important files to understand are: `model.py`, `dataset.py` and `bc.py`.

- The file `model.py` has the skeleton for the model (which you will have to complete in the following questions),

- The file `dataset.py` has the skeleton for the dataset the model is being trained with,

- and, `bc.py` will have all the structure for training the model with the dataset.


### 1.1 Dataset

We provide a pickle file with pre-collected expert demonstrations on CartPole from which to learn the policy $\pi$. The data has been collected from an expert policy on the environment, with the addition of a small amount of gaussian noise to the actions.

The pickle file contains a list of tuples of states and actions in `numpy` in the following way:

```
[(state s, action a), (state s, action a), (state s, action a), ...]
```

In the `dataset.py` file, we provide skeleton code for creating a custom dataset. The provided code shows how to load the file.

Your goal is to overwrite the `__getitem__` function in order to return a dictionary of tensors of the correct type.

Hint: Look in the `bc.py` file to understand how the dataset is used.

Answer the following questions:

- [**QUESTION 2 points]** Insert your code in the placeholder below.

In [5]:
# PLACEHOLDER TO INSERT YOUR __getitem__ method here
def __getitem__(self, index):
    
        item = self.data[index]
        state_tensor = torch.tensor(item[0]).float()
        if item[1] == 0:
            action_tensor= torch.tensor([1,0]).float()
        else:
            action_tensor= torch.tensor([0,1]).float()       
        state_action_dic = {'state': state_tensor, 'action': action_tensor}
        
        return state_action_dic
    
        raise NotImplementedError()

- **[QUESTION 2 points]** How big is the dataset provided?

**YOUR ANSWER HERE** The dataset consists of 99660 tuples of states and actions.

- **[QUESTION 2 points]** What is the dimensionality of $s$ and what range does each dimension of $s$ span?  I.e., how much of the state space does the expert data cover?

**YOUR ANSWER HERE** Each state has four values, the same as the previous assignment: the position of the Cart, the velocity of the Cart, Pole's angle, and Pole's angular velocity. The range that each dimension of $s$ span is provided as below:
![span.png](span.png)

As it can be seen, the state space is partially covered, and the coverage is not symmetric. The possible maximum and minimum  values for each state are as follows based on the Cartpole documentation:  

Cart Position: Min = -4.8, Max = 4.8. Cart Velocity: Min = -Inf, Max = Inf. Pole Angle: Min = -0.418 rad (-24 deg), Max = 0.418 rad (24 deg). Pole Angular Velocity: Min = -Inf, Max = Inf.

- **[QUESTION 2 points]** What are the dimensionalities and ranges of the action $a$ in the dataset (how much of the action space does the expert data cover)?

**YOUR ANSWER HERE** Although the question mentioned that the data had been collected, with the addition of a small amount of Gaussian noise to the actions $a$, the provided dataset only contains 2 discrete actions without any noise; going to the right (1) and going to the left (0).


### 1.2 Environment

Recall the state and action space of CartPole, from the previous assignment.

- **[QUESTION 2 points]** Considering the full state and action spaces, do you think the provided expert dataset has good coverage?  Why or why not? How might this impact the performance of our cloned policy?

**YOUR ANSWER HERE**  The action space is fully covered since it is discrete and contains only two actions, right(1) and left(0). In fact, the agent can learn and control the pole balancing only by commanding the Cart to go left or right. Regarding the states, it does not have good coverage for Cart's position and velocity, especially for negative values.  Furthermore, the coverage is not symmetric, which opposes the nature of the system.  However, in the previous assignment, we have shown that the position and velocity of the Cart are relatively less important compared to other states. Therefore, the provided coverage for Cart's position and velocity might not disrupt the learning. For the pole angle, we know that the episode termination happens when the pole angle is more than 12 degrees (0.2 rad). Again the coverage is not symmetric; It covers a relatively good domain of positive angles, but the coverage is not satisfactory for negative values. Finally, the domain covered by the pole's velocity is more symmetric compared to other states. But it could still be more comprehensive. If the expert dataset is not comprehensive enough, the agent might fail to choose the proper action in an unseen situation.

### 1.3 Model

The file `model.py` provides skeleton code for the model. Your goal is to create the architecture of the network by adding layers that map the input to output.

You will need to update the `__init__` method and the `forward` method.

The `select_action` method has already been written for you.  This should be used when running the policy in the environment, while the `forward` function should be used at training time.

- [**QUESTION 5 points]** Insert your code in the placeholder below.

In [4]:
# PLACEHOLDER TO INSERT YOUR MyModel class here

import torch
import torch.nn as nn
import torch.nn.functional as F

class MyModel(nn.Module):
    def __init__(self, state_size, action_size):        
        super(MyModel, self).__init__()
        self.fc1 = nn.Linear(state_size, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, action_size)

    def forward(self, x):
        y = torch.relu(self.fc1(x))
        y = torch.relu(self.fc2(y))
        
        ##### For BC we use this line of code as the output of the forward function since the output is the probability of choosing actions.
        y = torch.sigmoid(self.fc3(y))    

        ##### For DQN we use this line of code as the output of the forward function since the output is the Q(s,s). You can uncomment it if needed.
        #y = self.fc3(y)
        
        
        return y
        raise NotImplementedError()

    def select_action(self, state):
        self.eval()
        z = self.forward(state)
        self.train()
        return z.max(1)[1].view(1, 1).to(torch.long)


Answer the following questions:

- **[QUESTION 2 points]** What is the input of the network?

**YOUR ANSWER HERE** The input to the network are states. Basically, the purpose of BC/DQN is to find a policy that can choose the best action/estimate Q-values (to agian choose the best action) based on states.

- **[QUESTION 2 points]** What is the output?

**YOUR ANSWER HERE** As commented in the code above, the output of the network for the Behavioral Cloning section is action since we want to calculate the probability of choosing actions; thus, we have used the sigmoid function as an activation function in the last layer. However, the output of network for DQN section is Q-values because we want to estimate the Q-value for each state-action. As a result, the output of the network for the DQN section is a linear function without any activation function.

### 1.4 Training

The file `bc.py` is the entry point for training your behavioral cloning model. The skeleton and the main components are already there.

The missing parts for you to do are:

- Initializing the model
- Choosing a loss function
- Choosing an optimizer
- Playing with hyperparameters to train your model.

- [**QUESTION 5 points]** Insert your code in the placeholder below.

In [5]:
# PLACEHOLDER FOR YOUR CODE HER
# HOW DID YOU INITIALIZE YOUR MODEL, OPTIMIZER AND LOSS FUNCTIONS? PASTE HERE YOUR FINAL CODE
# NOTE: YOU CAN KEEP THE FOLLOWING LINES COMMENTED OUT, AS RUNNING THIS CELL WILL PROBABLY RESULT IN ERRORS

import gym
import torch
from eval_policy import eval_policy, device
from model import MyModel
from dataset import Dataset
import torch.nn as nn
import torch.optim as optim

BATCH_SIZE = 64
TOTAL_EPOCHS = 100
LEARNING_RATE = 1e-3
PRINT_INTERVAL = 500
TEST_INTERVAL = 2
MOMENTUM = 0.5

ENV_NAME = 'CartPole-v0'

dataset = Dataset(data_path="CartPole-v0_dataset.pkl".format(ENV_NAME))
dataloader = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE, num_workers=4)

env = gym.make(ENV_NAME)

# TODO INITIALIZE YOUR MODEL HERE
model = MyModel(4,2)


def train_behavioral_cloning():

    # TODO CHOOSE A OPTIMIZER AND A LOSS FUNCTION FOR TRAINING YOUR NETWORK
    optimizer = optim.Adam(model.parameters(), lr=0.0001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)
    loss_function =  nn.BCELoss()  
    
    gradient_steps = 0
        
    
    for epoch in range(1, TOTAL_EPOCHS + 1):
        for iteration, data in enumerate(dataloader):
            data = {k: v.to(device) for k, v in data.items()}
            output = model(data['state'])
            loss = loss_function(output, data["action"])
          
    
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
          
            if gradient_steps % PRINT_INTERVAL == 0:
                print('[epoch {:4d}/{}] [iter {:7d}] [loss {:.5f}]'
                      .format(epoch, TOTAL_EPOCHS, gradient_steps, loss.item()))
                
            gradient_steps += 1
    
        if epoch % TEST_INTERVAL == 0:
            score = eval_policy(policy=model, env=ENV_NAME)
            print('[Test on environment] [epoch {}/{}] [score {:.2f}]'
                  .format(epoch, TOTAL_EPOCHS, score))
    
    model_name = "behavioral_cloning_{}.pt".format(ENV_NAME)
    print('Saving model as {}'.format(model_name))
    torch.save(model.state_dict(), model_name)

if __name__ == "__main__":
    train_behavioral_cloning()

You can run your code by doing:

```
python3 bc.py
```

**During all of this assignment, the code in `eval_policy.py` will be your best friend.** At any time, you can test your model by giving as argument the path to the model weights and the environment name using the following command:

```
python3 eval_policy.py --model-path /path/to/model/weights --env ENV_NAME
````

**[QUESTION 2 points]** Did you manage to learn a good policy? How consistent is the reward you are getting?

**YOUR ANSWER HERE** As it can be seen, although the dataset provided does not cover the full domain of the states, the agent can learn very fast and achieve reward=200 after only four epochs. However, as discussed in the next section, "we are not always lucky enough to have access to a dataset of expert demonstrations. Replicating an expert policy suffers from compounding error. The policy $\pi$ only sees these "perfect" examples and has no knowledge on how to recover from states not visited by the expert. For this reason, as soon as it is presented with a state that is off the expert trajectory, it will perform poorly and will continue to deviate from a good trajectory without the possibility of recovering from errors."

## Task 2: Deep Q Learning

There are two main issues with the behavior cloning approach.

- First, we are not always lucky enough to have access to a dataset of expert demonstrations.
- Second, replicating an expert policy suffers from compounding error. The policy $\pi$ only sees these "perfect" examples and has no knowledge on how to recover from states not visited by the expert. For this reason, as soon as it is presented with a state that is off the expert trajectory, it will perform poorly and will continue to deviate from a good trajectory without the possibility of recovering from errors.

---
The second task consists in solving the environment from scratch, using RL, and most specifically the DQN algorithm, to learn a policy $\pi$.

For this task, familiarize yourself with the file `dqn.py`. We are going to re-use the file `model.py` for the model you created in the previous task.

Your task is very similar to the one in the previous assignment, to implement the Q-learning algorithm, but in this version, our Q-function is approximated with a neural network.

The algorithm (excerpted from Section 6.5 of [Sutton's book](http://incompleteideas.net/book/RLbook2018.pdf)) is given below:

![DQN algorithm](https://i.imgur.com/Mh4Uxta.png)

### 2.0 Think about your model...



**[QUESTION 2 points]** In DQN, we are using the same model as in task 1 for behavioral cloning. In both tasks the model receives as input the state and in both tasks the model outputs something that has the same dimensionality as the number of actions. These two outputs, though, represent very different things. What is each one representing?

**YOUR ANSWER HERE** As discussed in the BC section, the output of the Behavioral Cloning network is the probability of choosing action. This is why we used sigmoid as the activation function in the last layer of the network. However, in the DQN network, the output is Q-values; thus, we used the linear function without any activation function in the DQN network to achieve a better result since it is regarded as the regression problem and Q values could be possibly more than [0-1] domian. 

### 2.1 Update your Q-function

Complete the `optimize_model` function. This function receives as input a `state`, an `action`, the `next_state`, the `reward` and `done` representing the tuple $(s_t, a_t, s_{t+1}, r_t, done_t)$. Your task is to update your Q-function as shown in the [Atari DQN paper](https://arxiv.org/abs/1312.5602) environment. For now don't be concerned with the experience replay buffer. We'll get to that later.

![Loss function](https://i.imgur.com/tpTsV8m.png)

- [**QUESTION 8 points]** Insert your code in the placeholder below.

In [7]:
## PLACEHOLDER TO INSERT YOUR optimize_model function here:

def optimize_model(state, action, next_state, reward, done):
    
    # TODO given a tuple (s_t, a_t, s_{t+1}, r_t, done_t) update your model weights
    
    with torch.no_grad():
        q_next= target(next_state).float()
       
    
    out, inds = torch.max(q_next,dim=1)
    status  = torch.logical_not(done)+0
    y = (reward + GAMMA * status * out).view(-1,1)
    q = model(state)
    Q = torch.zeros(BATCH_SIZE,1)
    
    
    ##### BATCH_SIZE is BATCH_SIZE = 1 for the first part of the question, and BATCH_SIZE = 256 for the replay buffer section.
    for i in range(BATCH_SIZE):
        Q[i] = q[i,action[i].type(torch.LongTensor)] 
        
        
    loss = loss_func(y,Q)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

### 2.2 $\epsilon$-greedy strategy

You will need a strategy to explore your environment. The standard strategy is to use $\epsilon$-greedy. Implement it in the `choose_action` function template.

- [**QUESTION 5 points]** Insert your code in the placeholder below.

In [9]:
## PLACEHOLDER TO INSERT YOUR choose_action function here:

def choose_action(state, i_episode, test_mode=False):
    # TODO implement an epsilon-greedy strategy
    
    # We used Linear Annealing for decaying epsilon to reach faster convergence
    epsilon = max(0.02, 0.1 - 0.01*(i_episode/200)) 

    ee_tradeoff = np.random.random()

    if ee_tradeoff <= epsilon:
        action = torch.tensor(env.action_space.sample())  
    else:
        action = torch.argmax(model(torch.tensor(state).float()))
    
    return action.view(-1,1)
    raise NotImplementedError()


### 2.3 Train your model

Try to train a model in this way.

You can run your code by doing:

```
python3 dqn.py
```

**[QUESTION 2 points]** How many episodes does it take to learn (ie. reach a good reward)?

**YOUR ANSWER HERE** It has reached the maximum reward in a relatively stable situation after almost 1450 episodes. In some epochs, we see noisy behavior. This actually happens due to the stochastic nature of our training procedure. We will see in the next section that we can achieve faster convergence and more stable results using buffer replay. We have also used decaying epsilon, which will help have a more stable result in a shorter time.

### 2.4 Add the Experience Replay Buffer

If you read the DQN paper (and as you can see from the algorithm picture above), the authors make use of an experience replay buffer to learn faster. We provide an implementation in the file `replay_buffer.py`. Update the `train_reinforcement_learning` code to push a tuple to the replay buffer and to sample a batch for the `optimize_model` function.

**[QUESTION 5 points]** How does the replay buffer improve performances?

**YOUR ANSWER HERE** It is obvious that by using Experience Replay Buffer, the agent could reach the maximum reward in stable situation after only 600 epochs whcih is cpmparably faster than the precious section. Usig Experience Replay Buffer clearlsy shows that how benifiting drom past expericenc can effectivly increase the convergance time of the algortihm.
Advantages of Experience Replay Buffer:
1- More efficient use of previous experience, by learning with it multiple times. This is key when gaining real-world experience is costly, you can get full use of it. The Q-learning updates are incremental and do not converge quickly, so multiple passes with the same data is beneficial, especially when there is low variance in immediate outcomes (reward, next state) given the same state, action pair. 3- Better convergence behaviour when training a function approximator.


## Task 3: Extra

Ideas to experiment with:

Is $\epsilon$-greedy strategy the best strategy available? Why not trying something different.

Answer: As discussed earlier, we have used a decaying function in DQN Section to decrease epsilon after each espidoe and achieve faster convergence. The function that we have used is called Linear Annealing, and it is as follows:

    epsilon = max(0.02, 0.1 - 0.01*(i_episode/200)) 

